MIT License

Copyright (c) Microsoft Corporation. All rights reserved.

This notebook is adapted from Microsoft Learning mslearn-dp100 

Copyright (c) 2021 PyLadies Amsterdam, Alyona Galyeva

# Create batch pipeline

## Connect to your workspace

In [1]:
from azureml.core import Workspace
ws = Workspace.from_config()

## Provision inference compute

We'll need a compute context for the pipeline, so we'll use the following code to specify an Azure Machine Learning compute cluster (it will be created if it doesn't already exist).

Important: Cluster names must be globally unique names between 2 to 16 characters in length. Valid characters are letters, digits, and the - character.

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "mlopsbootcamp"

try:
    # Check for existing compute target
    inference_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS2_v2', max_nodes=2)
        inference_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        inference_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


Note: Compute instances and clusters are based on standard Azure virtual machine images. For this exercise, the Standard_DS11_v2 image is recommended to achieve the optimal balance of cost and performance. If your subscription has a quota that does not include this image, choose an alternative image; but bear in mind that a larger image may incur higher cost and a smaller image may not be sufficient to complete the tasks. Alternatively, ask your Azure administrator to extend your quota.

## Create a pipeline for batch inferencing

Now we're ready to define the pipeline we'll use for batch inferencing. Our pipeline will need Python code to perform the batch inferencing, so let's create a folder where we can keep all the files used by the pipeline:

In [3]:
import os
# Create a folder for the experiment files
experiment_folder = 'batch_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

batch_pipeline


In [4]:
# let's check what models are registered in our workspace and get a path to our model of choice
from azureml.core import Model
model_list = Model.list(ws)
#model_list
#model_path = Model.get_model_path('linear_regression', _workspace=ws)
model_path = Model.get_model_path('ridge', _workspace=ws)
print(model_list, model_path)

[Model(workspace=Workspace.create(name='mlops', subscription_id='1e7eebf7-7dfc-4e94-9219-13ee1fc677f1', resource_group='mlops_bootcamp'), name=ridge, id=ridge:1, version=1, tags={}, properties={}), Model(workspace=Workspace.create(name='mlops', subscription_id='1e7eebf7-7dfc-4e94-9219-13ee1fc677f1', resource_group='mlops_bootcamp'), name=linear_regression, id=linear_regression:1, version=1, tags={}, properties={})] azureml-models/ridge/1/ridge.pkl


In [5]:
# let's load our model and take a look what's inside
import joblib
model = joblib.load(model_path)
model


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encoder',
                                                  OneHotEncoder(sparse=False),
                                                  [2, 3, 4])])),
                ('regr_cv',
                 RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                                    estimator=Ridge(fit_intercept=False),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ffb6b85aac0>},
                                    scoring='neg_mean_squared_error',
                                    verbose=2))])

In [6]:
mini_batch = list()
for (dirpath, dirnames, filenames) in os.walk("batch-data"):
    mini_batch += [os.path.join(dirpath, file) for file in filenames]
for elem in mini_batch:
    print(elem)

batch-data/155.csv
batch-data/141.csv
batch-data/97.csv
batch-data/83.csv
batch-data/68.csv
batch-data/6.csv
batch-data/54.csv
batch-data/40.csv
batch-data/41.csv
batch-data/7.csv
batch-data/55.csv
batch-data/69.csv
batch-data/82.csv
batch-data/168.csv
batch-data/96.csv
batch-data/140.csv
batch-data/154.csv
batch-data/142.csv
batch-data/156.csv
batch-data/80.csv
batch-data/94.csv
batch-data/43.csv
batch-data/57.csv
batch-data/5.csv
batch-data/56.csv
batch-data/4.csv
batch-data/42.csv
batch-data/95.csv
batch-data/81.csv
batch-data/157.csv
batch-data/143.csv
batch-data/85.csv
batch-data/91.csv
batch-data/147.csv
batch-data/153.csv
batch-data/46.csv
batch-data/52.csv
batch-data/1.csv
batch-data/53.csv
batch-data/47.csv
batch-data/152.csv
batch-data/146.csv
batch-data/90.csv
batch-data/84.csv
batch-data/92.csv
batch-data/86.csv
batch-data/150.csv
batch-data/144.csv
batch-data/51.csv
batch-data/3.csv
batch-data/45.csv
batch-data/79.csv
batch-data/78.csv
batch-data/44.csv
batch-data/50.csv
b

In [7]:
import numpy as np
def run(mini_batch):
    # This runs for each batch
    resultList = []

    # process each file in the batch
    for f in mini_batch:
        # Read comma-delimited data into an array
        data = np.genfromtxt(f, delimiter=',')
        # Reshape into a 2-dimensional array for model input
        prediction = model.predict(data.reshape(1, -1))
        # Append prediction to results
        resultList.append("{}: {}".format(os.path.basename(f), prediction[0]))
    return resultList

In [8]:
result = run(mini_batch)
result

['155.csv: 5659.044500303951',
 '141.csv: 6453.884022171924',
 '97.csv: 6928.96696728335',
 '83.csv: 8170.145488584927',
 '68.csv: 7784.777413082775',
 '6.csv: 5341.164708222638',
 '54.csv: 6458.972931004642',
 '40.csv: 9276.099392888733',
 '41.csv: 9407.473666585682',
 '7.csv: 5741.673875782065',
 '55.csv: 6951.1647928471375',
 '69.csv: 7563.930699813918',
 '82.csv: 8008.6322375151885',
 '168.csv: 5752.488346113794',
 '96.csv: 7378.310182120897',
 '140.csv: 6841.30735529549',
 '154.csv: 5570.239233745771',
 '142.csv: 6410.685029936254',
 '156.csv: 5978.729051596117',
 '80.csv: 6894.690090067934',
 '94.csv: 7972.5660248626355',
 '43.csv: 9174.429523783443',
 '57.csv: 8081.889057192055',
 '5.csv: 5103.042140965036',
 '56.csv: 7466.464606342966',
 '4.csv: 5154.152751208154',
 '42.csv: 9452.699673951993',
 '95.csv: 7833.375136665221',
 '81.csv: 7559.413494159684',
 '157.csv: 6055.433855551955',
 '143.csv: 6261.073602448914',
 '85.csv: 8688.790401584694',
 '91.csv: 8712.092725796098',
 '14

Now we'll create a Python script to do the actual work, and save it in the pipeline folder:

In [9]:
%%writefile $experiment_folder/score.py
# windows users
# %%writefile $experiment_folder\score.py

import os
import numpy as np
from azureml.core import Model
import joblib

def init():
    # Runs when the pipeline step is initialized
    global model

    # load the model
    #model_path = Model.get_model_path('linear_regression')
    model_path = Model.get_model_path('ridge')
    model = joblib.load(model_path)

def run(mini_batch):
    # This runs for each batch
    resultList = []

    # process each file in the batch
    for f in mini_batch:
        # Read comma-delimited data into an array
        data = np.genfromtxt(f, delimiter=',')
        # Reshape into a 2-dimensional array for model input
        prediction = model.predict(data.reshape(1, -1))
        # Append prediction to results
        resultList.append("{}: {}".format(os.path.basename(f), prediction[0]))
    return resultList

Overwriting batch_pipeline/score.py


The pipeline will need an environment in which to run, so we'll create a Conda specification that includes the packages that the code uses.

In [10]:
%%writefile $experiment_folder/batch_environment.yml
# windows users
# %%writefile $experiment_folder\batch_environment.yml
name: batch_environment
dependencies:
- python=3.8
- numpy
- pandas
- scikit-learn
- pip:
  - azureml-core

Overwriting batch_pipeline/batch_environment.yml


Next we'll define a run context that includes the Conda environment.

In [11]:
from azureml.core import Environment
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# Create an Environment for the experiment
batch_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/batch_environment.yml")
batch_env.docker.base_image = DEFAULT_CPU_IMAGE
print('Configuration ready.')

Configuration ready.


You're going to use a pipeline to run the batch prediction script, generate predictions from the input data, and save the results as a text file in the output folder. To do this, you can use a ParallelRunStep, which enables the batch data to be processed in parallel and the results collated in a single output file named parallel_run_step.txt.

In [12]:
from datetime import datetime

from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.data import OutputFileDatasetConfig
from azureml.core.runconfig import DockerConfiguration

# # Get the batch dataset for input
batch_data_set = ws.datasets['batch-data']

# Set the output location
default_ds = ws.get_default_datastore()
output_dir = OutputFileDatasetConfig(name='inferences')

# Define the parallel run step step configuration
parallel_run_config = ParallelRunConfig(
    source_directory=experiment_folder,
    entry_script="score.py",
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=batch_env,
    compute_target=inference_cluster,
    node_count=2)

parallel_step_name = "batchscoring-" + datetime.now().strftime("%Y%m%d%H%M")

# Create the parallel run step
parallelrun_step = ParallelRunStep(
    name=parallel_step_name,
    parallel_run_config=parallel_run_config,
    inputs=[batch_data_set.as_named_input('batch_data')],
    output=output_dir,
    arguments=[],
    allow_reuse=True
)

print('Steps defined')

Steps defined
/Users/leticialunatlatelpa/opt/anaconda3/envs/mlops_train/lib/python3.8/site-packages/azureml/pipeline/core/_parallel_run_step_base.py:591: UserWarning: 
ParallelRunStep requires azureml-dataset-runtime[fuse] for file dataset.
Please add relevant package in CondaDependencies.
  warnings.warn(


Now it's time to put the step into a pipeline, and run it.

In [13]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline

# Create the pipeline
pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])

# Run the pipeline as an experiment
pipeline_run = Experiment(ws, 'nyc-energy-demand-batch').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)

dfece57173a5c74e963e10b59ffcfe3711de3ec847ca1f92b9fd5c3003a2_d.txt
[2021-07-18T17:48:09.921845] Entering job release
[2021-07-18T17:48:10.985819] Starting job release
[2021-07-18T17:48:10.987345] Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 372
[2021-07-18T17:48:10.987957] job release stage : upload_datastore starting...
[2021-07-18T17:48:10.988574] job release stage : start importing azureml.history._tracking in run_history_release.
[2021-07-18T17:48:10.992106] job release stage : copy_batchai_cached_logs starting...
[2021-07-18T17:48:10.992237] job release stage : copy_batchai_cached_logs completed...
[2021-07-18T17:48:10.994920] Entering context manager injector.
[2021-07-18T17:48:10.995202] job release stage : execute_job_release starting...
[2021-07-18T17:48:11.008535] job release stage : upload_datastore completed...
[2021-07-18T17:48:11.119586] job release stage : send_run_telemetry sta

'Finished'

When the pipeline has finished running, the resulting predictions will have been saved in the outputs of the experiment associated with the first (and only) step in the pipeline. You can retrieve it as follows:

In [14]:
import pandas as pd
import shutil

# Remove the local results folder if left over from a previous run
shutil.rmtree('batch-results', ignore_errors=True)

# Get the run for the first step and download its output
prediction_run = next(pipeline_run.get_children())
prediction_output = prediction_run.get_output_data('inferences')
prediction_output.download(local_path='batch-results')

# Traverse the folder hierarchy and find the results file
for root, dirs, files in os.walk('batch-results'):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# cleanup output format
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# Display the first 20 results
df.head(20)

,File,Prediction
0,1.csv,5759.551073
1,10.csv,7321.988511
2,100.csv,6114.465948
3,101.csv,6156.851918
4,102.csv,6294.679238
5,103.csv,6783.997580
6,104.csv,7328.934520
7,105.csv,7730.472261
8,106.csv,7639.333131
9,107.csv,8248.744239


## Publish the Pipeline and use its REST Interface

Now that you have a working pipeline for batch inferencing, you can publish it and use a REST endpoint to run it from an application.

In [16]:
#published_pipeline = pipeline_run.publish_pipeline(name='Linear_regression_batch_prediction_pipeline',
#                                                   description='Batch scoring using linear regression model',
#                                                   version='1.0')

published_pipeline = pipeline_run.publish_pipeline(name='Ridge_regression_batch_prediction_pipeline',
                                                   description='Batch scoring using ridge regression model',
                                                   version='1.0')
published_pipeline

Name,Id,Status,Endpoint
Ridge_regression_batch_prediction_pipeline,314c8917-07d1-4dde-bd08-01f664f454ca,Active,REST Endpoint


Note that the published pipeline has an endpoint, which you can see in the Azure portal. You can also find it as a property of the published pipeline object:

In [17]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

https://westeurope.api.azureml.ms/pipelines/v1.0/subscriptions/1e7eebf7-7dfc-4e94-9219-13ee1fc677f1/resourceGroups/mlops_bootcamp/providers/Microsoft.MachineLearningServices/workspaces/mlops/PipelineRuns/PipelineSubmit/314c8917-07d1-4dde-bd08-01f664f454ca


To use the endpoint, client applications need to make a REST call over HTTP. This request must be authenticated, so an authorization header is required. To test this out, we'll use the authorization header from your current connection to your Azure workspace, which you can get using the following code:

Note: A real application would require a service principal with which to be authenticated.

In [18]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print('Authentication header ready.')

Authentication header ready.


Once published, you can use this endpoint to initiate a batch inferencing job, as shown in the following example code:

In [19]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "nyc-energy-demand-batch"})
run_id = response.json()["Id"]
run_id

'e47fe786-42be-40fc-bf4d-ef7b25099200'

You can also schedule the published pipeline to have it run automatically, as shown in the following example code:

In [ ]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

#weekly = ScheduleRecurrence(frequency='Week', interval=1)
daily = ScheduleRecurrence(frequency='Day', interval=1)
daily = 
pipeline_schedule = Schedule.create(ws, name='Weekly Predictions',
                                        description='batch inferencing',
                                        pipeline_id=published_pipeline.id,
                                        experiment_name='Batch_Prediction',
                                        recurrence=daily)

IMPORTANT: Remove inference cluster if you do not plan to work on exercises immediately!!!